# Probamos la funcion para saber si alguien esta en casa o no

In [8]:
import pandas as pd
import tensorflow as tf

data_list1 = [[39.0, 116.0, '2008-10-23 13:53:88', '000'],
             [39.984198, 116.319322, '2008-10-23 13:53:06', '000'],
             [39.984094, 116.319236, '2008-10-23 13:53:11', '000'],
             [39.0, 116.0, '2008-10-23 13:599:99', '000'],
             [39.984123, 116.319437, '2010-23 13:53:44', '000'],
             [39.984123, 116.319437, '2008-23 13:53:55', '000'],
             [39.0, 117.0, '2008-10-23 13:53', '001'],
             [39.0, 116.0, '2008-10-23 13:53:12', '002'],
             [39.984324, 116.319421, '2008-10-23 13:53:111', '001']]
data_columns  = ['lat', 'lng', 'datetime', 'uid']



In [9]:

df_test1 = pd.DataFrame(data_list1, columns=data_columns)
df_test1['at_home'] = 0
df_test1

,lat,lng,datetime,uid,at_home
0,39.000000,116.000000,2008-10-23 13:53:88,000,0
1,39.984198,116.319322,2008-10-23 13:53:06,000,0
2,39.984094,116.319236,2008-10-23 13:53:11,000,0
3,39.000000,116.000000,2008-10-23 13:599:99,000,0
4,39.984123,116.319437,2010-23 13:53:44,000,0
5,39.984123,116.319437,2008-23 13:53:55,000,0
6,39.000000,117.000000,2008-10-23 13:53,001,0
7,39.000000,116.000000,2008-10-23 13:53:12,002,0
8,39.984324,116.319421,2008-10-23 13:53:111,001,0


In [10]:
data_list2 = [[39.0, 116.0, '000'],
             [39.984324, 116.319421, '001']]
data_columns  = ['lat', 'lng', 'uid']
df_test2 = pd.DataFrame(data_list2, columns=data_columns)
df_test2['at_home'] = 1
df_test2


,lat,lng,uid,at_home
0,39.000000,116.000000,000,1
1,39.984324,116.319421,001,1


In [11]:

df_test1 = pd.merge(df_test1, df_test2, on=['lat','lng','uid'], how='left')
df_test1

,lat,lng,datetime,uid,at_home_x,at_home_y
0,39.000000,116.000000,2008-10-23 13:53:88,000,0,1.0
1,39.984198,116.319322,2008-10-23 13:53:06,000,0,NaN
2,39.984094,116.319236,2008-10-23 13:53:11,000,0,NaN
3,39.000000,116.000000,2008-10-23 13:599:99,000,0,1.0
4,39.984123,116.319437,2010-23 13:53:44,000,0,NaN
5,39.984123,116.319437,2008-23 13:53:55,000,0,NaN
6,39.000000,117.000000,2008-10-23 13:53,001,0,NaN
7,39.000000,116.000000,2008-10-23 13:53:12,002,0,NaN
8,39.984324,116.319421,2008-10-23 13:53:111,001,0,1.0


In [12]:

df_test1['at_home'] = df_test1['at_home_y'].fillna(df_test1['at_home_x'])  
df_test1 = df_test1.drop(['at_home_x','at_home_y'], axis=1)
df_test1



,lat,lng,datetime,uid,at_home
0,39.000000,116.000000,2008-10-23 13:53:88,000,1.0
1,39.984198,116.319322,2008-10-23 13:53:06,000,0.0
2,39.984094,116.319236,2008-10-23 13:53:11,000,0.0
3,39.000000,116.000000,2008-10-23 13:599:99,000,1.0
4,39.984123,116.319437,2010-23 13:53:44,000,0.0
5,39.984123,116.319437,2008-23 13:53:55,000,0.0
6,39.000000,117.000000,2008-10-23 13:53,001,0.0
7,39.000000,116.000000,2008-10-23 13:53:12,002,0.0
8,39.984324,116.319421,2008-10-23 13:53:111,001,1.0


# Probamos a crear un modelo con Tensorflow, y pasar la data a tensores

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[3]),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation = 'sigmoid')
  ])

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

model = build_model()


In [16]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


In [1]:
import tensorflow as tf
import pandas as pd
import skmob
from skmob.measures.individual import home_location
from skmob.io.file import load_geolife_trajectories, read, write

json_path= "../../Data/gps_data.json"
other_df = read (json_path)

In [2]:
other_df.head()

,lat,lng,datetime,uid
0,39.982375,116.320442,2009-09-19 07:11:37,120
1,39.982380,116.320448,2009-09-19 07:11:48,120
2,39.982380,116.320455,2009-09-19 07:11:50,120
3,39.982397,116.320455,2009-09-19 07:11:52,120
4,39.982425,116.320460,2009-09-19 07:11:54,120


In [6]:
import numpy as np
other_df['ts'] = other_df.datetime.values.astype(np.int64) // 10**9

other_df.head()

,lat,lng,datetime,uid,ts
0,39.982375,116.320442,2009-09-19 07:11:37,120,1253344297
1,39.982380,116.320448,2009-09-19 07:11:48,120,1253344308
2,39.982380,116.320455,2009-09-19 07:11:50,120,1253344310
3,39.982397,116.320455,2009-09-19 07:11:52,120,1253344312
4,39.982425,116.320460,2009-09-19 07:11:54,120,1253344314


In [8]:
other_df = pd.DataFrame(other_df)
other_df.describe()

,lat,lng,uid,ts
count,92830.000000,92830.000000,92830.000000,9.283000e+04
mean,38.559663,115.764088,121.804212,1.250047e+09
std,4.675042,2.255154,0.751236,4.495135e+06
min,22.534653,108.726840,120.000000,1.242494e+09
25%,39.964671,116.231612,122.000000,1.244921e+09
50%,39.976559,116.398177,122.000000,1.251904e+09
75%,40.131145,116.474284,122.000000,1.253721e+09
max,42.551380,117.957961,123.000000,1.255298e+09


In [13]:

other_df = other_df.drop('datetime', axis = 1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(other_df) 
other_df.loc[:,:] = scaled_values
other_df.tail()

,lat,lng,uid,ts
8205,0.584057,0.024237,1.0,0.903905
8206,0.584059,0.024239,1.0,0.903905
8207,0.584061,0.024239,1.0,0.903906
8208,0.584074,0.024239,1.0,0.903908
8209,0.584073,0.024239,1.0,0.903908
